In [5]:
# Install the package in editable mode
!uv pip install -e ..

Using Python 3.11.14 environment at: /home/gjeannin/artefactual/.venv
Resolved 9 packages in 120ms                                         
   Building artefactual @ file:///home/gjeannin/artefactual            
Resolved 9 packages in 120ms                                         
   Building artefactual @ file:///home/gjeannin/artefactual            
   Building artefactual @ file:///home/gjeannin/artefactual    
   Building artefactual @ file:///home/gjeannin/artefactual    
      Built artefactual @ file:///home/gjeannin/artefactual    
Prepared 1 package in 393ms                                              
Uninstalled 1 package in 1ms
Installed 1 package in 2ms(from file:///home/gjeannin/artefa
 ~ artefactual==0.1.0 (from file:///home/gjeannin/artefactual)
      Built artefactual @ file:///home/gjeannin/artefactual
Prepared 1 package in 393ms                                              
Uninstalled 1 package in 1ms
Installed 1 package in 2ms(from file:///home/gjeannin/artefa
 ~ 

In [ ]:
import os
import pathlib

from vllm import LLM, SamplingParams

from artefactual.scoring import EPR, WEPR

project_root = pathlib.Path.cwd().parent.resolve()

## 1. Generate Response with vLLM

We use vLLM to generate a response for a single question.
We request logprobs to compute entropy-based scores.


In [4]:
# Initialize vLLM
# Ensure you have access to the model or change the model name to one you have access to
model_name = "mistralai/Ministral-8B-Instruct-2410"

# Set CUDA_VISIBLE_DEVICES to use a free GPU (e.g., GPU 2)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

llm = LLM(model=model_name)

# Define the prompt
prompts = ["Who is the president of the United States?"]

# Define sampling parameters
# We need logprobs for WEPR/EPR computation
sampling_params = SamplingParams(temperature=0.0, logprobs=20, max_tokens=50)

# Generate response
outputs = llm.generate(prompts, sampling_params)

# Print the generated text
generated_text = outputs[0].outputs[0].text

INFO 12-24 13:22:57 [utils.py:253] non-default args: {'seed': None, 'disable_log_stats': True, 'model': 'mistralai/Ministral-8B-Instruct-2410'}
INFO 12-24 13:22:58 [model.py:637] Resolved architecture: MistralForCausalLM
INFO 12-24 13:22:58 [model.py:1750] Using max model len 131072
INFO 12-24 13:22:58 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 12-24 13:22:58 [model.py:637] Resolved architecture: MistralForCausalLM
INFO 12-24 13:22:58 [model.py:1750] Using max model len 131072
INFO 12-24 13:22:58 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.


[2025-12-24 13:22:58] INFO tekken.py:176: Adding special tokens <SPECIAL_20>, ..., <SPECIAL_999>
[2025-12-24 13:22:58] INFO tekken.py:540: Vocab size: 150000
[2025-12-24 13:22:58] INFO tekken.py:544: Cutting vocab to first 130072 tokens.
[2025-12-24 13:22:58] INFO tekken.py:540: Vocab size: 150000
[2025-12-24 13:22:58] INFO tekken.py:544: Cutting vocab to first 130072 tokens.


(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:00 [core.py:93] Initializing a V1 LLM engine (v0.12.0) with config: model='mistralai/Ministral-8B-Instruct-2410', speculative_config=None, tokenizer='mistralai/Ministral-8B-Instruct-2410', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_parser='', reasoning_parser_plugin='', enable_in_reasoning=False), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None, kv_cache_metrics=

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.24s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.24s/it]
(EngineCore_DP0 pid=4023822) 
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.24s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.24s/it]
(EngineCore_DP0 pid=4023822) 


(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:06 [default_loader.py:308] Loading weights took 3.45 seconds
INFO 12-24 13:23:06 [default_loader.py:308] Loading weights took 3.45 seconds
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:06 [gpu_model_runner.py:3549] Model loading took 14.9694 GiB memory and 4.328569 seconds
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:06 [gpu_model_runner.py:3549] Model loading took 14.9694 GiB memory and 4.328569 seconds
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:11 [backends.py:655] Using cache directory: /home/gjeannin/.cache/vllm/torch_compile_cache/905ce91c15/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:11 [backends.py:715] Dynamo bytecode transform time: 4.07 s
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:11 [backends.py:655] Using cache directory: /home/gjeannin/.cache/vllm/torch_compile_cache/905ce91c15/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:11 [backend

...

(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:20 [gpu_model_runner.py:4466] Graph capturing finished in 4 secs, took 0.59 GiB
INFO 12-24 13:23:20 [gpu_model_runner.py:4466] Graph capturing finished in 4 secs, took 0.59 GiB
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:20 [core.py:254] init engine (profile, create kv cache, warmup model) took 13.77 seconds
(EngineCore_DP0 pid=4023822) INFO 12-24 13:23:20 [core.py:254] init engine (profile, create kv cache, warmup model) took 13.77 seconds


(EngineCore_DP0 pid=4023822) [2025-12-24 13:23:20] INFO tekken.py:176: Adding special tokens <SPECIAL_20>, ..., <SPECIAL_999>
(EngineCore_DP0 pid=4023822) [2025-12-24 13:23:20] INFO tekken.py:540: Vocab size: 150000
(EngineCore_DP0 pid=4023822) [2025-12-24 13:23:20] INFO tekken.py:544: Cutting vocab to first 130072 tokens.
[2025-12-24 13:23:20] INFO tekken.py:176: Adding special tokens <SPECIAL_20>, ..., <SPECIAL_999>
(EngineCore_DP0 pid=4023822) [2025-12-24 13:23:20] INFO tekken.py:540: Vocab size: 150000
(EngineCore_DP0 pid=4023822) [2025-12-24 13:23:20] INFO tekken.py:544: Cutting vocab to first 130072 tokens.


INFO 12-24 13:23:22 [llm.py:343] Supported tasks: ['generate']
Generating response...
Generating response...


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.24s/it, est. speed input: 1.21 toks/s, output: 3.64 toks/s]

Generated Text:  The current president of the United States is Joseph R. Biden Jr. He took office on January 20, 2021.


## 2. Initialize WEPR with Custom Weights

We initialize the `WEPR` class by pointing it to the specific weights file for the Ministral model.

In [7]:
# Define path to the weights file
# Assuming the notebook is in 'examples/' and the file is in 'src/artefactual/data/'
weights_path = project_root / "src/artefactual/data/weights_ministral.json"

# Check if file exists
if not weights_path.exists():
    msg = f"Weights file not found at: {weights_path}"
    raise FileNotFoundError(msg)

# Initialize WEPR
# The 'model' argument can be a path to a JSON file containing the weights
wepr = WEPR(model=str(weights_path))
epr = EPR(model=str(project_root / "src/artefactual/data/calibration_ministral.json"))

## 3. Compute Scores for All Samples

We compute WEPR and EPR scores for all 10 samples.


[RequestOutput(request_id=0, prompt='Who is the president of the United States?', prompt_token_ids=[1, 31500, 1395, 1278, 10233, 1307, 1278, 4857, 6086, 1063], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text=' The current president of the United States is Joseph R. Biden Jr. He took office on January 20, 2021.', token_ids=[1531, 3519, 10233, 1307, 1278, 4857, 6086, 1395, 11493, 1434, 1046, 66224, 16791, 1046, 2182, 5244, 9750, 1408, 6866, 1032, 1050, 1048, 1044, 1032, 1050, 1048, 1050, 1049, 1046, 2], cumulative_logprob=-5.926176261856085, logprobs=[{1531: Logprob(logprob=-1.5777908563613892, rank=1, decoded_token=' The'), 14424: Logprob(logprob=-2.4215407371520996, rank=2, decoded_token=' Joe'), 20398: Logprob(logprob=-2.8590407371520996, rank=3, decoded_token=' Who'), 24898: Logprob(logprob=-2.8590407371520996, rank=4, decoded_token=' Donald'), 1319: Logprob(logprob=-3.1402907371520996, rank=5, decoded_token=' ('), 103

In [9]:
# Compute scores for the single output
# The compute method expects a list of outputs or a single output object
# It returns a list of scores

wepr_scores = wepr.compute(outputs)
epr_scores = epr.compute(outputs)

epr_token_level_scores = epr.compute_token_scores(outputs)
wepr_token_level_scores = wepr.compute_token_scores(outputs)

wepr_score = wepr_scores[0]
epr_score = epr_scores[0]


WEPR Score: 0.8100503566291674
EPR Score: 0.9999999999996603
WEPR Token-level Scores: [0.99790025 0.55006045 0.05929238 0.03018122 0.03145245 0.03145734
 0.03169614 0.02835398 0.13182984 0.03610038 0.0304431  0.03156283
 0.02588933 0.04263151 0.05623298 0.13289703 0.02965864 0.02741074
 0.03178211 0.03148381 0.03145764 0.03131408 0.03088118 0.03147761
 0.03147975 0.03148025 0.03146403 0.03129065 0.06211783 0.9062832 ]
EPR Token-level Scores: [3.0561693e+00 2.0080497e+00 1.0562013e+00 7.7853334e-01 4.6324534e-03
 1.0769630e-02 6.4461357e-03 1.6148485e-01 1.9778041e+00 3.6181253e-01
 2.1605838e-02 1.4578647e-02 1.6165379e-01 8.1536806e-01 4.7030860e-01
 1.6716672e+00 2.9711109e-02 7.3007055e-02 1.6918281e-02 3.0829341e-04
 4.4105260e-04 2.3310145e-03 6.8612481e-03 2.9114416e-04 4.5976598e-05
 3.4797147e-05 3.8942546e-04 2.3475820e-03 1.1031432e+00 2.4985414e+00]


## 4. Results Analysis

We analyze the scores to determine if the response is likely a hallucination.
Threshold is 0.5.


In [10]:
threshold = 0.5


if wepr_score > threshold:
    pass

Prompt: Who is the president of the United States?
Response:  The current president of the United States is Joseph R. Biden Jr. He took office on January 20, 2021.
------------------------------
WEPR Score: 0.8101
EPR Score:  1.0000
------------------------------
Prediction: HALLUCINATION (Uncertain)
